In [1]:
import sys
sys.path.append("..")

from utils import *
from trainer import *
import scanpy as sc
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import argparse
import pandas as pd
import scipy.sparse as sp
seed=2024
fix_seed(seed)

In [2]:
adata1_path='/mnt/sdb/tyh/Data_SpatialGlue/Dataset14_Simulation2/adata_RNA.h5ad'
adata2_path='/mnt/sdb/tyh/Data_SpatialGlue/Dataset14_Simulation2/adata_ADT.h5ad'

In [3]:
adata_omics1,adata_omics2=pre_process(adata1_path,adata2_path,data_type='Simulation',
                                      spatial_neighbors=4,
                                      omics1_neighbors=10,
                                      omics2_neighbors=10,
                                      radius=0.2
                                     )

../utils.py:37: ImplicitModificationWarning: Setting element `.obsm['adj_spatial']` of view, initializing view as actual.
  adata_omics1.obsm['adj_spatial'] = spatial_adj


In [4]:
train(
    adata_omics1, adata_omics2,
    save_path='./1.h5ad',
    n_cluster=5,
    lr=0.001,
    weight_decay=0.1,
    Epoch=5000,
    device='cuda:1'
)

Training Progress, loss: 1.3198,rec1: 1.0788,rec2: 0.1182, mmd1: 0.0001,mmd2: 0.0000, sp: 0.1225, dec: 0.0003: 100%|█| 5000/5000 [01:07<00:00, 74
R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


In [5]:
data=sc.read_h5ad('./1.h5ad')

In [6]:
tmp_label=data.obsm['spfac']
labels=[]
for i in tmp_label:
    if np.sum(i)==0:
        labels.append(4)
    else:
        labels.append(np.argwhere(i==1)[0][0])
        
labels=np.array(labels)

In [7]:
print(adjusted_rand_score(labels, data.obs['mclust']))

1.0
